In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np

def conv(z, h, omega_lambda, omega_matter):

    H0=100*h
#conversion Mpc --> km
    conv1=1./(1e6*206265*1.496e8)
#conversion de s --> Gyr
    conv2=3600*24*365.25*1e9
    H0=H0*conv1*conv2

    c1=np.sqrt(omega_lambda*(1.+z)**(-3.))+np.sqrt((omega_lambda*(1.+z)**(-3.))+omega_matter)
    ln=np.log(c1/np.sqrt(omega_matter))
    c2=2./(3.*np.sqrt(omega_lambda))
        
    time = (1./H0)*c2*ln

    return time


In [22]:
import matplotlib.pyplot as plt
import h5py
import rotation as rot
import sphviewer as sph
import numpy as np

h=0.732
G = 4.299e-6
a0=1
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter
vector2 = ('M31','MW','M33')
vector = ('00','01','03')

snapshot=range(273,10,-1)
for isnap in ([497]+snapshot):
    snap = h5py.File('/home/ornela/python/ornella/outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')
    #print isnap
    for i in range(0,1):
        cm   = snap['subhalo_0'+ str('%s' %vector[i])+ '/Center'].value
        r200 = snap['subhalo_0'+ str('%s' %vector[i])+ '/R200'].value
        aexp = snap['subhalo_0'+ str('%s' %vector[i])+ '/Time'].value

        pstr = snap['subhalo_0'+ str('%s'%vector[i]) + '/Str/Coordinates'].value
        pgas = snap['subhalo_0'+ str('%s'%vector[i]) + '/Gas/Coordinates'].value
        masa_str = snap['subhalo_0'+ str('%s' %vector[i]) + '/Str/Masses'].value
        masa_gas = snap['subhalo_0'+ str('%s' %vector[i]) + '/Gas/Masses'].value
        vel  = snap['subhalo_0'+ str('%s' %vector[i])+ '/Str/Velocities'].value

        z = a0/aexp - 1.
        Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
        t = conv(z, h, omega_lambda, omega_matter)
    #---aca paso las coordenadas respecto al centro de la galaxia------
        xstr = (pstr[:,0]-cm[0])*aexp/h
        ystr = (pstr[:,1]-cm[1])*aexp/h
        zstr = (pstr[:,2]-cm[2])*aexp/h
        r = np.sqrt(xstr**2+ystr**2+zstr**2)
        
        xgas = (pgas[:,0]-cm[0])*aexp
        ygas = (pgas[:,1]-cm[1])*aexp
        zgas = (pgas[:,2]-cm[2])*aexp
        r_gas = np.sqrt(xgas**2+ygas**2+zgas**2)

        v_x = vel[:,0] *np.sqrt(aexp) + Ht *xstr/1000.
        v_y = vel[:,1] *np.sqrt(aexp) + Ht *ystr/1000.
        v_z = vel[:,2] *np.sqrt(aexp) + Ht *zstr/1000.

    #----------------------masas----------------------------
        masa_str = masa_str/h

        r200 = r200*aexp/h
        rgal=0.15*r200

        limit = np.where(r<rgal)
        r_sort = np.sort(r[limit])
        r_indice = np.argsort(r[limit])

        Mc_str = cumsum((masa_str[limit])[r_indice])
        M_gal = Mc_str[-1]

    #------------------ calculamos M90------------------------------------------
        razon = Mc_str/M_gal
        noventa, = np.where(razon < 0.9)
        cincuenta, = np.where(razon < 0.5)

        r90 = r_sort[noventa]
        r50 = r_sort[cincuenta]
        M90 = Mc_str[noventa]                

    #------------------masa total en r200 (Mvir)--------------------------         
        corte,=np.where(r<r90[-1])
        veloc,=np.where(r<r50[-1])

    #----------componentes de la velocidad del centro de masa------------
        vxcm = sum(masa_str[veloc]*v_x[veloc])/sum(masa_str[veloc])
        vycm = sum(masa_str[veloc]*v_y[veloc])/sum(masa_str[veloc])
        vzcm = sum(masa_str[veloc]*v_z[veloc])/sum(masa_str[veloc])

    #----- velocidades de las estrellas respecto del centro de masa de la galaxia---------
        vx = vel[:,0] - vxcm
        vy = vel[:,1] - vycm
        vz = vel[:,2] - vzcm
        
        if isnap == 497:
            e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(masa_str,xstr,ystr,zstr,vx,vy,vz,r90[-1])

    ##posiciones de partículas que se quiere graficar
    ##como lo de arriba me da los versores hago las posiciones con esto 

        xn = e1x*xgas + e1y*ygas + e1z*zgas
        yn = e2x*xgas + e2y*ygas + e2z*zgas
        zn = e3x*xgas + e3y*ygas + e3z*zgas
        vxn = e1x*vx + e1y*vy + e1z*vz
        vyn = e2x*vx + e2y*vy + e2z*vz
        vzn = e3x*vx + e3y*vy + e3z*vz

        pos=np.ndarray([3,np.size(xn)])
        pos[0,:]=xn
        pos[1,:]=yn
        pos[2,:]=zn

    #---------------------generador del gráfico-----------------
        rl=50   
        corte,=np.where((xn<rl) & (yn<rl) & (zn<rl) & (xn>-rl) & (yn>-rl) & (zn>-rl))

    #-----rango que tiene la escala  de colores-----
        vmin=1
        vmax=7

    # ----escala de colores que te guste (http://matplotlib.org/examples/color/colormaps_reference.html)---
        cmap='gnuplot'

    #-----grafico-----------
        fig=plt.figure(1,figsize=(8,8))
        fig.subplots_adjust(bottom=0.12, left =0.16, right = 0.95, top = 0.95)

        particles=sph.Particles(pos[:,corte],masa_gas[corte]*1e10,nb=50)
        escena=sph.Scene(particles)
        escena.update_camera(r='infinity',x=0,y=0,z=0,extent=[-rl,rl,-rl,rl])
        rend=sph.Render(escena)
        extent=escena.get_extent()
        rend.set_logscale()

        ax1=fig.add_subplot(111)
        plt.imshow(rend.get_image(),extent=extent,origin='lower',cmap=cmap, vmin=vmin, vmax= vmax)
        #plt.axis('off')
        #ax1.set_title('Gadget - '+ str('%s'%vector2[i]), fontsize=18)
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
        #plt.xlabel('$x$'+' $[kpc]$', fontsize=26)
        #plt.ylabel('$y$'+' $[kpc]$', fontsize=30)
        ax1.minorticks_on()
        ax1.tick_params( labelsize=24)
        ax1.tick_params('both', length=5, width=1.8,which='minor')
        ax1.tick_params('both', length=8, width=1.8,which='major')
        plt.text(-47, 45,'GADGET-'+'%s'%vector2[i], fontsize=24, color='yellow', ha='left', va='center') 
                 #bbox={'facecolor':'black', 'alpha':1, 'pad':2})
        plt.annotate("",xy=(20, -47), xycoords='data',xytext=(45, -47),textcoords='data',
                      ha='center', va='center', 
                     arrowprops=dict(arrowstyle="|-|", connectionstyle='arc3', color ='white', lw=2.5))

        plt.text(32.5, -47, '25 kpc', fontsize=24, color='white', ha='center', va='bottom')
        plt.text(-45, -45, str('%.3f'%t)+'Gyr', fontsize=24, color='white', ha='left', va='center')


        '''particles=sph.Particles(pos[:,corte],masa_str[corte]*1e10,nb=50)
        escena=sph.Scene(particles)
        escena.update_camera(r='infinity',x=0,y=0,z=0,extent=[-rl,rl,-rl,rl], t=90)
        rend=sph.Render(escena)
        extent=escena.get_extent()
        rend.set_logscale()

        ax2=fig.add_subplot(212)
        plt.imshow(rend.get_image(),extent=extent,origin='lower',cmap=cmap, vmin=vmin, vmax= vmax)
        #plt.axis('off')
        plt.xlabel('$x$'+' $[kpc]$', fontsize=30)
        plt.ylabel('$z$'+' $[kpc]$', fontsize=30)
        ax2.minorticks_on()
        ax2.tick_params( labelsize=24)
        ax2.tick_params('both', length=5, width=1.8,which='minor')
        ax2.tick_params('both', length=8, width=1.8,which='major')
        #plt.text(12.5, 12.5, str('%.3f'%t)+'Gyr', fontsize=22, color='white', ha='left', va='top')''' 


        fig.savefig('/home/ornela/python/sem2/M31_gas/'+str('%03d'%isnap)+'-M31.png', facecolor= 'k', edgecolor='k', frameon= 'False',bbox_inches='tight',pad_inches=0, dpi = 100)
        #fig.savefig('/home/ornela/Escritorio/mi_tesis/Figures/starGD_'+str('%s'%vector2[i])+'.png', dpi = 100)
        #plt.show()
        plt.clf()

In [21]:
x = range(0,10)
y = [47]
print y+x

[47, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
